In [36]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [37]:
spark = SparkSession.builder \
    .master("spark://dezoom-hw.asia-southeast1-b.c.homeworks-448912.internal:7077") \
    .appName('test') \
    .getOrCreate()

In [38]:
spark

In [39]:
#read parquet file
df_green = spark.read.parquet('data/pq/green/*/*')

In [40]:
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [41]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [42]:
df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [43]:
common_colums = []

yellow_columns = set(df_yellow.columns)

for col in df_green.columns:
    if col in yellow_columns:
        common_colums.append(col)


In [44]:
df_green_sel = df_green \
    .select(common_colums) \
    .withColumn('service_type', F.lit('green'))

In [45]:
df_yellow_sel = df_yellow \
    .select(common_colums) \
    .withColumn('service_type', F.lit('yellow'))

In [46]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [47]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 1734051|
|      yellow|28482270|
+------------+--------+



In [48]:
df_trips_data.columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge',
 'service_type']

In [49]:
df_trips_data.registerTempTable('trips_data')

In [50]:
df_result = spark.sql("""
SELECT 
    -- Revenue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_monthly_passenger_count,
    AVG(trip_distance) AS avg_monthly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [51]:
df_result.coalesce(1).write.parquet('data/report/rev/')

25/03/21 11:13:32 WARN TaskSetManager: Lost task 7.0 in stage 19.0 (TID 97) (10.148.0.2 executor 0): org.apache.spark.SparkException: Parquet column cannot be converted in file file:///home/thatlaconic/DEzoomcamp_activities/Module_5/data/pq/yellow/2024/10/part-00003-b1dfa74e-8182-41d7-9435-9147209837f1-c000.snappy.parquet. Column: [passenger_count], Expected: int, Found: INT64.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:855)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:290)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:131)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:593)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.columnartorow_nextBatch_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedCla

Py4JJavaError: An error occurred while calling o356.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 8 in stage 19.0 failed 4 times, most recent failure: Lost task 8.3 in stage 19.0 (TID 104) (10.148.0.2 executor 0): org.apache.spark.SparkException: Parquet column cannot be converted in file file:///home/thatlaconic/DEzoomcamp_activities/Module_5/data/pq/yellow/2024/10/part-00002-b1dfa74e-8182-41d7-9435-9147209837f1-c000.snappy.parquet. Column: [passenger_count], Expected: int, Found: INT64.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:855)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:290)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:131)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:593)
	at org.apache.spark.sql.catalyst.expressions.